# analysis of inertial partcles in daily averaged field of NWES
In this notebook we analyse the effect of MR advection on the trajectory of plastics.   
The particles were released in the daily flowfield in the NWES from the NWES dataset (https://doi.org/10.48670/moi-00054)    
They are released the 1st of every month from 1 september 2023 to 1 march 2024 and simulated for 30 days, where there   
location is saved very hour.  

We analyse 3 types of particles:
1. tracer particles
2. randomly displaced tracer particles
4. SMMR particles with flexible Rep

We compare 2-3 to 1 to study effect of MR vs tracer particles

We analyse:
- trajectory length 
- seperation distance
- denstity distribution 

And then compare to the results of the simulation with daily resolution

In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

# import needed packages
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import colormaps
import matplotlib.cm as cm
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from decimal import Decimal
import hexbin_functions as hexfunc
import h3
from shapely.geometry import shape, Point
from datetime import datetime, timedelta
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/simulations")
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from analysis_functions import  make_PDF, make_lognormal_PDF, Haversine
from analysis_functions_xr import trajectory_length, calc_tidal_av


from helper import create_filelist

# set plotscale
plt.style.use('../python_style_Meike.mplstyle')
colorlist=np.array(['black','grey','firebrick'])#'orange','green','red','navy','pink'])
lineslist = ['-','-','--','--',':']


In [ ]:

#set directories and template filenames
base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'


basefile_MR_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}_tres_daily'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}.zarr')

basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_tres_daily'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}.zarr')


basefile_tracer = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}_tres_daily'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_{land_handling}.zarr')
basefile_tracer_random = (base_directory + 'tracer_random/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}_tres_daily'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_d{d:04d}_{land_handling}.zarr')



particle_types = ['tracer','tracer_random','inertial_SM_drag_Rep','inertial_SM_Rep_constant']#,'inertial_drag_Rep']
basefiles = {'tracer':basefile_tracer,
             'tracer_random':basefile_tracer_random,
             'inertial_SM_drag_Rep':basefile_MR_Rep_drag,
              'inertial_SM_Rep_constant':basefile_Rep_constant,
             'inertial_drag_Rep':basefile_MR_Rep_drag

             }

legend_names = {'tracer':'tracer',
             'tracer_random':'displaced tracer',
             'inertial_SM_drag_Rep':'flexible Re$_p$',
             'inertial_drag_Rep':'MR'
             }



In [ ]:
# set constants 
Rearth = 6371 * 10**3 # in k,
deg2rad = np.pi / 180.

#for simulation
runtime =  timedelta(days=30)# timedelta(days=10)
loc = 'NWES'
runtime=timedelta(30)
nparticles = 51548# 88347 # 52511
land_handling = 'anti_beaching'
coriolis = True
Replist=[100,450,1000] # [0,10,100,450,1000]
starttimes = np.array([datetime(2023, 9, 1, 0, 0, 0, 0),
                       datetime(2023, 10, 1, 0, 0, 0, 0),
                       datetime(2023, 11, 1, 0, 0, 0, 0) ,
                       datetime(2023, 12, 1, 0, 0, 0, 0) ,
                       datetime(2024, 1, 1, 0, 0, 0, 0),
                       datetime(2024, 2, 1, 0, 0, 0, 0)])


chunck_time = 100
years = np.array([2023,2024])
months = np.array([1,2,3,4,5,6,7,8,9,10,11,12])
                      
#for particle
displacement = 300 #m


B = 0.68
tau = 2994.76 #2759.97

Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.2, 0.8, 3)) # Magma(np.linspace(0.2, 0.9, 5))

## Import data

In [ ]:
data ={}
for pt in particle_types:
    data[pt]={}
    for coriolis in [True]:
        data[pt][coriolis]={}
        if(pt == 'inertial_SM_Rep_constant'):
            for Rep in Replist:
                data[pt][coriolis][Rep]={}
        else:
            data[pt][coriolis][None]={}

            
for pt in particle_types:
    for coriolis in [True]:
        for starttime in starttimes:
            endtime = starttime + runtime 
            date = f'{starttime.year:04d}/{starttime.month:02d}'
            if(pt == 'inertial_SM_Rep_constant'):
                for Rep in Replist:
                    file = basefiles[pt].format(loc=loc,
                                                                    y_s=starttime.year,
                                                                    m_s=starttime.month,
                                                                    d_s=starttime.day,
                                                                    y_e=endtime.year,
                                                                    m_e=endtime.month,
                                                                    d_e=endtime.day,
                                                                    B = int(B * 1000), 
                                                                    tau = int(tau ),
                                                                    land_handling = land_handling, 
                                                                    coriolis = coriolis,
                                                                    particle_type = pt,
                                                                    Rep = Rep,
                                                                    d = displacement)
                    # print(file)
                    ds = xr.open_dataset(file,
                                            engine='zarr',
                                            chunks={'trajectory':nparticles, 'obs':chunck_time},
                                            drop_variables=['B','tau','z'],
                                            decode_times=False) #,decode_cf=False)

                    data[pt][coriolis][Rep][date]= ds 
            else:
                file = basefiles[pt].format(loc=loc,
                                                                y_s=starttime.year,
                                                                m_s=starttime.month,
                                                                d_s=starttime.day,
                                                                y_e=endtime.year,
                                                                m_e=endtime.month,
                                                                d_e=endtime.day,
                                                                B = int(B * 1000), 
                                                                tau = int(tau ),
                                                                land_handling = land_handling, 
                                                                coriolis = coriolis,
                                                                Rep = None,
                                                    
                                                                particle_type = pt,
                                                                d = displacement)
                    # print(file)
                ds = xr.open_dataset(file,
                                        engine='zarr',
                                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                                        drop_variables=['B','tau','z'],
                                        decode_times=False) #,decode_cf=False)

                data[pt][coriolis][None][date]= ds 


In [ ]:
ds_test = xr.open_dataset('/storage/shared/oceanparcels/output_data/data_Meike/NWES/NWES_daily_hex_release_new.nc')
print(ds_test)

In [ ]:
# plot selection of trajectories
Blues = colormaps['Blues']

fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.3, 0.95, 5))
colorlist=['k','grey','cornflowerblue','blue']
markers = ['-','-','--','-.']
year=2023
month=9
date = f'{year:04d}/{month:02d}'
legend= []
coriolis = True

ax.coastlines()
idstart =np.random.randint(0,nparticles)#25#8813#171#141#8969#25#25 #600#100#25#6008 # np.random.randint(0,nparticles)#22#3748 #568#25680#47584#98
print(idstart)
idstep = 1
idend = idstart+1 * idstep
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')
for pt, color, marker in zip(particle_types,colorlist,markers):


        



        # ax.legend(particle_types,loc=(1,0.2))
        # pt = 'inertial_SM_Rep_constant'
        if(pt == 'inertial_SM_Rep_constant'):
                for Rep, color2 in zip(Replist,Magmalist):
                        ax.plot(data[pt][coriolis][Rep][date].lon[idstart:idend:idstep,0::1].T,
                                data[pt][coriolis][Rep][date].lat[idstart:idend:idstep,0::1].T,
                                marker,
                                color=color2);

                        legend.append('Re$_p$={:d}'.format(Rep)) #simtype[pt]+
        else:
                ax.plot(data[pt][coriolis][None][date].lon[idstart,0::1].T,
                data[pt][coriolis][None][date].lat[idstart,0::1].T,
                marker,
                color=color,zorder=20);
                legend.append(legend_names[pt])
ax.legend(legend,fontsize=18, handlelength = 1.5,loc=(1,0.))
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
        linewidth=0, color='gray', alpha=0.5, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}
# ax.set_title(simtype[pt])

# ax.set_xlim(-15,10)
# ax.set_ylim(42,62)
# ax.set_xlim(-14.6,-10.5)
# ax.set_ylim(54.4,56.1)
# if(pt == 'inertial_SM_Rep_constant'):
#         fig.tight_layout()
#         fig.savefig('../figures/poster/trajectory_ID25_rep.pdf')
# conclusion Rep = 0, 1, 10 clearly different (but themselves very similar), 100 di

In [ ]:

print(np.argmin(np.fabs(data['tracer'][True][None][date].lon[:,0].values+14.497238)))
print(np.argmin(np.fabs(data['tracer'][True][None][date].lat[:,0].values-54.736893)))


In [ ]:
id = 33020 
print(data['tracer'][True][None][date].lon[id,0].values)
print(data['tracer'][True][None][date].lat[id,0].values)

In [ ]:
select =data['tracer'][True][None][date].where(np.fabs(data['tracer'][True][None][date].lon[0,:]+3.361572)<10)
select = select.where(np.fabs(data['tracer'][True][None][date].lat[0,:]-47.10494)<10)
print(select.time[0].values)

## Along track displacement

In [ ]:
# along track distance
for pt in particle_types: # loop over all particle types
    for starttime in starttimes: # loop over all starttime
        year = starttime.year
        month = starttime.month
        date = f'{year:04d}/{month:02d}'
        if(pt == 'inertial_SM_Rep_constant'):
            for Rep in Replist:
                ds =data[pt][coriolis][Rep][date]
                traj_length = trajectory_length(ds.lon, ds.lat)
                data[pt][coriolis][Rep][date] = ds.assign(traj_length = traj_length)
               
    
        else:
            ds=data[pt][coriolis][None][date]
            traj_length = trajectory_length(ds.lon, ds.lat)
            data[pt][coriolis][None][date] = ds.assign(traj_length = traj_length)
          
            


    


In [ ]:
#  plot PDF of total trajectory length
fig,ax=plt.subplots()

Tmax = 720

legend=[]
Tmax=720
for pt, color, marker in zip(particle_types,colorlist,markers):




    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist): 
            legend.append(' Re$_p$ ='+' {:d}'.format(Rep))
            data_arrays = list(data[pt][coriolis][Rep].values())
            combined_data = xr.concat(data_arrays, dim="starting_days")

            average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)

            bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),401, norm=True)
            ax.plot(bins,pdf,marker,color=color2)
    else:
        legend.append(legend_names[pt])
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days")

        average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)

        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),401, norm=True)


        ax.plot(bins,pdf,marker,color=color,zorder=20)
# ax.set_title(simtype[pt])
ax.legend(legend)
ax.set_ylabel("PDF")
ax.set_xlabel("trajectory length [km]")


    #tidal_av
   


# for particle_type in particle_types:
#     bins, pdf = make_PDF(arrays[particle_type+'_tidal_av'],401, norm=True)
#     ax.plot(bins,pdf,'--',color=color_particles[particle_type])
#     legend.append(particle_type+' tidal av')



In [ ]:
# plot PDF difference trajecory length and rest of particles
i =0
fig, ax = plt.subplots()
data_arrays_tracer = list(data['tracer'][coriolis][None].values())
combined_data_tracer = xr.concat(data_arrays_tracer, dim="starting_days")
legend = []
Tmax = 720
for pt, color, marker in zip(particle_types[1:],colorlist[1:],markers[1:]):





    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            
            diff = combined_data_Rep.traj_length - combined_data_tracer.traj_length
            average_data = diff.mean(dim="starting_days",skipna=True)
            bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
            ax.plot(bins,pdf,marker,color=color2, zorder = i)
            legend.append('Re$_p$ ='+' {:d}'.format(Rep))
            i-=5
    else:
        print(pt)
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days") 
        diff = combined_data.traj_length - combined_data_tracer.traj_length
        average_data = diff.mean(dim="starting_days",skipna=True)
        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
        ax.plot(bins,pdf,marker,color=color)
        legend.append(legend_names[pt])

    # ax.set_title(simtype[pt])
    ax.legend(legend,handlelength = 1.5)#,loc=(1,0.2))
    ax.set_ylabel("PDF")
    ax.set_xlabel("trajecory length difference to tracer [km]")
    ax.set_title('daily velocity fields',fontsize=18)
    # ax.set_xlim(-50,50)
    # ax.set_yscale('log')

## Relative dispersion

In [ ]:
# # EVALULATE IF YOU WANT TO REMOVE DISTANCE CALCULATION
# for starttime in starttimes:
#     print(starttime)
#     year = starttime.year
#     month = starttime.month
#     date = f'{year:04d}/{month:02d}'
#     for pt_a, pt_b in [['tracer_random','tracer'],['inertial_SM_drag_Rep','tracer']]:
#         data[pt_a][coriolis][date]=data[pt_a][coriolis][date].drop_vars('dist_'+pt_a+'_'+pt_b)

In [ ]:
# Relative dispersion


for starttime in starttimes:
    print(starttime)
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}' #'inertial_SM_drag_Rep','inertial_Rep_constant','inertial_SM_Rep_constant']
    ds_tracer = data['tracer'][coriolis][None][date]
    for pt in particle_types[1:]:
        if(pt == 'inertial_SM_Rep_constant'):
            for Rep in Replist:
                ds_Rep = data[pt][coriolis][Rep][date]
                dist= Haversine(ds_Rep['lon'],ds_Rep['lat'],
                                    ds_tracer['lon'],ds_tracer['lat'])
                data[pt][coriolis][Rep][date]=data[pt][coriolis][Rep][date].assign(dist = dist)
                data[pt][coriolis][Rep][date]=data[pt][coriolis][Rep][date].rename({'dist':'dist_to_tracer'})

            
        else:
            ds = data[pt][coriolis][None][date]
            dist= Haversine(ds['lon'],ds['lat'],
                                    ds_tracer['lon'],ds_tracer['lat'])
            data[pt][coriolis][None][date]=data[pt][coriolis][None][date].assign(dist = dist)
            data[pt][coriolis][None][date]=data[pt][coriolis][None][date].rename({'dist':'dist_to_tracer'})

            

        

In [ ]:
# plot distribution final distance between particles
# slow manifold finds bigger seperation distance (as expected because more oscillations)
tfinal = 799
markerlist=np.array(['-','--','-.',':','--','--','-.',':'])

i=0
Tmax=720
# cmap = plt.colormaps['magma']
# magma_colors = cmap(np.linspace(0.2, 0.9,4))
fig,ax=plt.subplots()
legend=[]
for pt,color,marker in zip(particle_types[1:],colorlist[1:],markerlist[:]):   

    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            mean=combined_data_Rep['dist_to_tracer'].mean(dim='trajectory',skipna=True)
            # std =combined_data_Rep['dist_to_tracer'].std(dim='trajectory',skipna=True)
            average_mean = mean.mean(dim="starting_days",skipna=True)
            average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
            
            tlist=np.arange(0,Tmax,1)/24
        
            # ax.errorbar(tlist[:720], average_mean[:720],average_std[:720], fmt='-', color=color)
            ax.plot(tlist[:720], average_mean[:720],'-.', color=color2)
            ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color2,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
        
            legend.append('Re$_p=$'+ ' {:d}'.format(Rep))
    else:
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days") 
        mean=combined_data['dist_to_tracer'].mean(dim='trajectory',skipna=True)
        # std =combined_data_Rep['dist_to_tracer'].std(dim='trajectory',skipna=True)
        average_mean = mean.mean(dim="starting_days",skipna=True)
        average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
        
        tlist=np.arange(0,Tmax,1)/24
        # ax.errorbar(tlist[:720], average_mean[:720],average_std[:720], fmt='-', color=color)
        print(marker)
        ax.plot(tlist[:720], average_mean[:720],marker, color=color,zorder=20)
        ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
    
        legend.append(legend_names[pt])

    ax.legend(legend,fontsize=18,handlelength=1.5)
    ax.set_xlabel('t [days]')
    ax.set_ylabel('distance to tracer [km]')
    ax.set_title('daily velocity fields',fontsize=18)
    # ax.set_yscale('log')
    # ax.set_xscale('log')

In [ ]:
# plot distribution final distance between particles
tfinal = 719
fig, ax = plt.subplots()
ref = 'tracer'
i=0
legend=[]
    # for pt_a, pt_b in [['tracer_random','tracer'],['inertial_drag_Rep','tracer'],['inertial_SM_drag_Rep','tracer'],['inertial_drag_Rep','inertial_SM_drag_Rep']]:
# pt_a ='inertial_SM_drag_Rep'
# pt_b= 'tracer'


particle_types_compare = [pt for pt in particle_types if pt != ref]
Tmax=400



for pt, color, line in zip(particle_types_compare, colorlist[1:],lineslist[1:]):
    # Extract DataArray objects from the dictionary
    data_arrays = list(data[pt][coriolis].values())

    # Combine DataArray objects along a new dimension
    combined_data = xr.concat(data_arrays, dim="starting_days") 
    diff = combined_data[f'dist_{pt}_{ref}'][:,:,tfinal] 
    array = diff.values.flatten()
    bins,pdf = make_PDF(array,1001,norm=True,vmin=0,vmax=100)#,min=0,max=6)
    # Calculate the average along the new dimension


    # bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten() ,101, norm=True, vmin = -100, vmax =100)
    ax.plot(bins,pdf,line,color=color)
    # ax.set_xscale('log')
    # ax.set_yscale('log')
    ax.set_xlim(0,10)
    ax.set_xlabel('trajectory length differenc [km] ')
    ax.set_ylabel('PDF')



In [ ]:
# plot distribution final distance between particles
# slow manifold finds bigger seperation distance (as expected because more oscillations)
tfinal = 720
Tmax = 720
markerlist=np.array(['-','--','-.',':','-','--','-.',':'])
fig,ax=plt.subplots()
i=1
legend=[]


for pt_a, pt_b in [['tracer_random','tracer'],['inertial_SM_drag_Rep','tracer']]:
    data_arrays = list(data[pt_a][coriolis].values())
    combined_data = xr.concat(data_arrays, dim="starting_days") 
    mean=combined_data['dist_'+pt_a+'_'+pt_b].mean(dim='trajectory',skipna=True)
    average_mean = mean.mean(dim="starting_days",skipna=True)
    average_sd_error = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
  
    tlist=np.arange(0,Tmax,1)/24

    # ax.errorbar(tlist[:720], average_mean[:720],average_sd_error[:720], fmt=markerlist[i], color=colorlist[i])
    ax.plot(tlist[:720], average_mean[:720],markerlist[i], color=colorlist[i])
    ax.fill_between(tlist[:720], average_mean[:720]+average_sd_error[:720],  average_mean[:720]-average_sd_error[:720],color=colorlist[i],alpha=0.2,label='_nolegend_')#,alhpa=0.2)
    # ax.plot(tlist[:720], values[:720], lineslist[i+1], color=colorlist[i+1])
    # ax.plot(tlist[:720], average_mean[:720],markerlist[i], color=colorlist[i])

    legend.append(f'{legend_names[pt_a]} - {legend_names[pt_b]}')


    
    
    i+=1
ax.legend(legend)#,loc=(1,0.2), fontsize=15)
ax.set_xlabel('t [days]')
ax.set_ylabel('distance [km]')
fig.tight_layout()
# fig.savefig('../figures/full_simulations/daily_field_av_seperation_distance.pdf')
# ax.set_yscale('log')
# ax.set_xscale('log')
# ax.axvline((5*24-12)/24)



In [ ]:
# calculate relative dispersion averaged over multiple months

fig,ax=plt.subplots()
tend =  700
tstart = 1
Tmax=720
tlist=np.arange(1,Tmax,1)
legend=[]
colors=['firebrick','green','navy']


i=1

for pt_a, pt_b in [['tracer_random','tracer'],['inertial_SM_drag_Rep','tracer']]:    
    Dtotal = np.zeros(800)
    data_arrays = list(data[pt_a][coriolis].values())
    combined_data = xr.concat(data_arrays, dim="starting_days") 
    dsquared = (combined_data['dist_'+pt_a+'_'+pt_b]**2).mean(dim='trajectory',skipna=True)
    dsquared_averaged = dsquared.mean(dim='starting_days',skipna=True)
    ax.plot(tlist,dsquared_averaged[1:Tmax],color=colorlist[i])
    # for starttime in starttimes:
    #     year = starttime.year
    #     month = starttime.month
    #     date = f'{year:04d}/{month:02d}'
    #     d = (data[pt_a][coriolis][date]['dist_'+pt_a+'_'+pt_b]**2).mean(dim='trajectory',skipna=True).values
    #     Dtotal += d 

        
    legend.append(f'{legend_names[pt_a]} {legend_names[pt_b]}')
    # legend.append(f'{pt_a} {pt_b} tidal_av')

    # Dtotal/=starttimes.size
   
    # ax.plot(tlist[tstart:tend],Dtotal[tstart:tend],color=colorlist[i+1])

    i+=1
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylabel('Relative Dispersion [m$^2$]')
ax.set_xlabel('time [hours]')
ax.legend(legend,loc=(1,0.2),fontsize=15)


# distribution analysis

In [ ]:
starttime = starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'
Tmax = 720-1
for pt, color in zip(particle_types, colorlist):
    fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
    ax.scatter(data[pt][coriolis][date].lon[:,Tmax],data[pt][coriolis][date].lat[:,Tmax],marker='o',s=2,alpha=0.1,color=color)
    ax.coastlines()
    ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                linewidth=0, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size': 20}
    gl.ylabel_style =  {'size': 20}

# not really anything visible from these plots

In [ ]:
startime = starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'
fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
pt = 'tracer_random'

ds =data[pt][coriolis][None][date]

dist_filled = ds['dist_to_tracer'].ffill('obs')

# Select last value along the desired axis
last_value = dist_filled[:,-1]

# Normalize by its maximum non-NaN value
sec_in_day=(60*60*24)
max_day=(ds['time']-ds['time'][:,0]).max(dim='obs', skipna=True)/sec_in_day


normalized1 = last_value / max_day


scp =ax.scatter(ds.lon[:,0],ds.lat[:,0],c=normalized1,s=0.4, vmin=0,vmax=1,cmap='Blues_r') # ds['dist_to_tracer'].ffill('obs')[:,-1]
fig.colorbar(scp,label='distance to tracer \nper day [km/day]',fraction=0.03,extend='max')
ax.coastlines()
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}
ax.set_title('displaced tracer')

fig.tight_layout()

In [ ]:
startime = starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'
fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
pt = 'inertial_SM_drag_Rep'

ds =data[pt][coriolis][None][date]

dist_filled = ds['dist_to_tracer'].ffill('obs')

# Select last value along the desired axis
last_value = dist_filled[:,-1]

# Normalize by its maximum non-NaN value
sec_in_day=(60*60*24)
max_day=(ds['time']-ds['time'][:,0]).max(dim='obs', skipna=True)/sec_in_day


normalized1 = last_value / max_day


scp =ax.scatter(ds.lon[:,0],ds.lat[:,0],c=normalized1,s=0.4, vmin=0,vmax=1,cmap='Blues_r') # ds['dist_to_tracer'].ffill('obs')[:,-1]
fig.colorbar(scp,label='distance to tracer \nper day [km/day]',fraction=0.03,extend='max')
ax.coastlines()
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}
ax.set_title('flexible Re$_p$')

fig.tight_layout()

startime = starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'
fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
pt = 'tracer_random'

ds =data[pt][coriolis][None][date]

dist_filled = ds['dist_to_tracer'].ffill('obs')

# Select last value along the desired axis
last_value = dist_filled[:,-1]

# Normalize by its maximum non-NaN value
sec_in_day=(60*60*24)
max_day=(ds['time']-ds['time'][:,0]).max(dim='obs', skipna=True)/sec_in_day


normalized1 = last_value / max_day


scp =ax.scatter(ds.lon[:,0],ds.lat[:,0],c=normalized1,s=0.4, vmin=0,vmax=5,cmap='magma') # ds['dist_to_tracer'].ffill('obs')[:,-1]
fig.colorbar(scp,label='distance to tracer \nper day [km/day]',fraction=0.03,extend='max')
ax.coastlines()
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}
ax.set_title('displaced tracer')

fig.tight_layout()

In [ ]:
# show final shapshot
startime = starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'
pt = 'inertial_SM_Rep_constant'
Tmax=719#5*24-12#719


i=0
for pt_a, pt_b in [['tracer_random','tracer'],['inertial_SM_drag_Rep','tracer']]:   
    fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
    sc = ax.scatter(data[pt_a][coriolis][date].lon[::,Tmax],data[pt_a][coriolis][date].lat[::,Tmax],marker='o',s=0.4,alpha=1,c=data[pt_a][coriolis][date]['dist_'+pt_a+'_'+pt_b].ffill(dim='trajectory')[::,Tmax],cmap='magma_r',vmin=0,vmax=20)
    cbar = fig.colorbar(sc,fraction=0.03,extend='max')
    cbar.ax.tick_params(labelsize=15) 
    cbar.set_label(label='Distance to tracer after 30 days',size=18)
    ax.coastlines()
    ax.set_title(f'{legend_names[pt_a]} - {legend_names[pt_b]}',fontsize=18)
    ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                linewidth=1, color='gray', alpha=0, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size': 15}
    gl.ylabel_style =  {'size': 15}
    fig.tight_layout()
    fig.savefig(f'../figures/full_simulations/daily_field_scatter_distance_{pt_a}_tracer_30_day_sep_2023.pdf')

In [ ]:
# make plot wher you sort of normalize the particle trajectoreis with distance to tracer for randomly displaced particles (so to really see where the MR is very differnt)
# show final shapshot
starttime = starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'
pt = 'inertial_SM_drag_Rep'#tracer'#'inertial_SM_drag_Rep'#'inertial_SM_Rep_constant'
print(particle_types[2])
Tmax=719#5*24-12#719


i=0

fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
coloring = data['inertial_SM_drag_Rep'][coriolis][date]['dist_inertial_SM_drag_Rep_tracer'].ffill(dim='trajectory')[::,Tmax] -data['tracer_random'][coriolis][date]['dist_tracer_random_tracer'].ffill(dim='trajectory')[::,Tmax]
sc = ax.scatter(data[pt][coriolis][date].lon[::,Tmax],data[pt][coriolis][date].lat[::,Tmax],marker='o',s=0.4,alpha=1,c=coloring,cmap='RdBu_r',vmin=-20,vmax=20)
cbar = fig.colorbar(sc,fraction=0.03,extend='both')
cbar.ax.tick_params(labelsize=15) 
cbar.set_label(label='$\\Delta_{\\mathrm{tracer}}^{\\mathrm{SM MR}} - \\Delta_{\\mathrm{tracer}}^{\\mathrm{displaced}}$ after 30 days',size=18)
ax.coastlines()
ax.set_title('position SM MR 1 October 2023',fontsize=15)
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0.1, color='gray', alpha=0, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 15}
gl.ylabel_style =  {'size': 15}
fig.tight_layout()
fig.savefig('../figures/full_simulations/daily_field_scatter_distance_SMMR-random_tracer_30_day_sep_2023.pdf')

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.sort(coloring))

In [ ]:
hex_res = 6
particles_per_bin=7**(6-hex_res)
sim_domain = { 
    "type":"Polygon",
    "coordinates": [
   [[9.977004051208496-0.5,61.28188705444336-0.5],
     [-15.996014595031738+0.5,61.28188705444336-0.5],
     [-15.996014595031738+0.5,46.00364303588867+0.5],
     [9.977004051208496-0.5,46.00364303588867+0.5],
     [9.977004051208496-0.5,61.28188705444336-0.5]]
     ]}
hexagons = h3.polygon_to_cells(h3.geo_to_h3shape(sim_domain), hex_res)
hexbin_grid = hexfunc.hexGrid(hexagons, h3_res=hex_res)
hexbin_grid.process_hexagons()

In [ ]:
startime = starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'
count={}
for pt in particle_types:
    ds = data[pt][coriolis][date]
    count[pt]=hexbin_grid.count_2d(ds.lon[:, Tmax].values,ds.lat[:,Tmax].values, normalize=False)/particles_per_bin

In [ ]:
fig, ax = plt.subplots(figsize = (60,30),subplot_kw={"projection": ccrs.PlateCarree()})
Tmax=719

sm =hexbin_grid.pcolorhex(cmap = 'Blues',counts=count['inertial_SM_drag_Rep'],minnorm=0,maxnorm=5, ax = ax, draw_edges=True)
ax.coastlines()
# ax.add_feature(cart.feature.LAND, edgecolor="black", color="lightgray",zorder=100)

In [ ]:
fig, ax = plt.subplots(figsize = (60,30),subplot_kw={"projection": ccrs.PlateCarree()})
Tmax=719

sm =hexbin_grid.pcolorhex(cmap = 'RdBu',counts=(count['tracer_random']-count['tracer']),minnorm=-2,maxnorm=2, ax = ax, draw_edges=True)
ax.coastlines()
# ax.add_feature(cart.feature.LAND, edgecolor="black", color="lightgray",zorder=100)

In [ ]:
fig, ax = plt.subplots(figsize = (60,30),subplot_kw={"projection": ccrs.PlateCarree()})
Tmax=719

sm =hexbin_grid.pcolorhex(cmap = 'RdBu',counts=(count['inertial_SM_drag_Rep']-count['tracer']),minnorm=-2,maxnorm=2, ax = ax, draw_edges=True)
ax.coastlines()
# ax.add_feature(cart.feature.LAND, edgecolor="black", color="lightgray",zorder=100)

In [ ]:
probability_set_time={}
starttime=starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'

pt = particle_types[0]
Tlist = [0,24*15-1,719]
rep = 0
for t in Tlist:
    print(t)
    ds =data[pt][coriolis][date]
    probability_set_time[t] = (hexbin_grid.count_2d(ds.lon[:, t].values,ds.lat[:,t].values, normalize=False))

In [ ]:
fig,ax = plt.subplots()
fig2,ax2 = plt.subplots()
# norm = 
tlegend=[]
for t in Tlist:
    bins, pdf = make_PDF(probability_set_time[t][probability_set_time[0]>0],nbins=261,norm=True,vmin=-0.5,vmax=260.5)
    ax.plot(bins[1:]/particles_per_bin,pdf[1:],'--o')
    dx = bins[2]-bins[1]
    ax2.plot(bins[1:]/particles_per_bin,np.cumsum(pdf[1:]*dx))
    tlegend.append(f'T = {(t+1)/24:.02f} days')
ax.axvline(12,color='grey',zorder=-20)
ax.legend(tlegend)
ax.set_xlim(-0.1,5)
ax.set_xlabel('particle density')
ax.set_ylabel('PDF')
ax.axvline(1,color='grey',zorder=-20)
ax2.axvline(1,color='grey',zorder=-20)
ax2.legend(tlegend)
ax2.set_xscale('log')

In [ ]:
probability_set = {}
Tmax=719

for pt in particle_types:
    probability_set[pt]={}
    for coriolis in [True]:
        probability_set[pt][coriolis]={}
for starttime in starttimes:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'
    print(date)
    for pt in particle_types:
        for coriolis in [True]:
            ds =data[pt][coriolis][date]
            
            probability_set[pt][coriolis][date] =(hexbin_grid.count_2d(ds.lon[:, Tmax].values,ds.lat[:,Tmax].values, normalize=False))
 
           

In [ ]:
fig, ax = plt.subplots()
fig2,ax2 = plt.subplots()
coriolis = True
starttime=starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'
legend=[]
for pt, color in zip(particle_types, colorlist):
    bins, pdf = make_PDF(probability_set[pt][coriolis][date],nbins=261,norm=True,vmin=-0.5,vmax=260.5)
    ax.plot(bins[1:]/particles_per_bin,pdf[1:],'--o',color=color)
    dx = bins[2]-bins[1]
    ax2.plot(bins[1:]/particles_per_bin,np.cumsum(pdf[1:]*dx),color=color)
    legend.append(legend_names[pt])

ax.legend(legend)
ax.set_xlabel('particle density')
ax.set_ylabel('PDF')
ax.axvline(1,color='grey',zorder=-20)
ax2.axvline(1,color='grey',zorder=-20)
ax2.legend(legend)
ax2.set_xlabel('particle density')
ax2.set_ylabel('CDF')
ax.set_xlim(-0.1,5)
ax2.set_xscale('log')